In [91]:


# The game
# 
WHITE = 'w'
BLACK = 'b'
EMPTY = '_'

WHITE_X = 'X'
BLACK_X = 'O'
EMPTY_X = ' ' # '·'
VALID_X = '?'

WIDTH  = 8
HEIGHT = 8

BLANK_BOARD = [EMPTY for x in range(WIDTH * HEIGHT)]

class Reversi:
    def __init__(self):
        self._turn = WHITE
        self._board = BLANK_BOARD[:]
        self._complete = False
        self._moves = []
        
        # Starting configuration
        #
        self.place(3, 3, checked = False)
        self.place(3, 4, checked = False)
        self.place(4, 4, checked = False)
        self.place(4, 3, checked = False)
    
    def stats(self):
        whitepoints = 0
        blackpoints = 0
        empty = 0
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if self._board[(x * WIDTH) + y] == WHITE:
                    whitepoints += 1
                elif self._board[(x * WIDTH) + y] == BLACK:
                    blackpoints += 1
                elif self._board[(x * WIDTH) + y] == EMPTY:
                    empty += 1
        return (self._complete or empty == 0, whitepoints, blackpoints, empty)

    
    def place(self, x, y, checked = True):
        if checked and (x, y) not in self.all_valid_placements():
            raise Exception("Invalid move!")
            
        if checked:
            self._moves.append((self._turn, (x, y), self._board[:]))

            opponent_colour = self.opposite_color(self._turn)
            opponent_offsets = self.adjacent_offsets_matching(x, y, opponent_colour)
            for (ox, oy) in opponent_offsets:
                #print ("X = %d, Y = %d -> OX = %d, OY = %d"%(x, y, ox, oy))
                paths = self.find_path_to(x, y, ox, oy, self._turn)
                if paths:
                    for (px, py) in paths:
                        # Set all pathways to our colour
                        #
                        self._board[(px * WIDTH) + py] = self._turn
        
        
        self._board[(x * WIDTH) + y] = self._turn
        self._turn = BLACK if self._turn == WHITE else WHITE
      
    def skip(self):
        if len(self.all_valid_placements()) > 0:
            raise Exception("Invalid attempt to skip (valid moves found)!")
        self._turn = BLACK if self._turn == WHITE else WHITE
        
        # Check we not in deadlock (ie next move is also going to want to skip)
        if len(self.all_valid_placements()) == 0:
            #print("Game deadlocked, finishing")
            self._complete = True
    
    def get_at(self, x, y):
        return self._board[(x * WIDTH) + y]
    
    def on_board(self, x, y):
        return x < WIDTH and x >= 0 and y < HEIGHT and y >= 0
    
    def opposite_color(self, colour):
        return WHITE if colour == BLACK else BLACK
    
    def adjacent_offsets_matching(self, x, y, colour):
        tests = [
            (-1, -1), (0, -1), (+1, -1),
            (-1,  0),          (+1,  0),
            (-1, +1), (0, +1), (+1, +1)
        ]
        offsets = [(tx, ty) for (tx, ty) in tests 
                   if self.on_board((tx + x), (ty + y)) and self.get_at(tx + x, ty + y) == colour]
        return offsets
        
    def all_valid_placements(self):
        opponent_colour = self.opposite_color(self._turn)
        result = []
        for x in range(WIDTH):
            for y in range(HEIGHT):
                if self._board[(x * WIDTH) + y] == EMPTY:
                    opponent_offsets = self.adjacent_offsets_matching(x, y, opponent_colour)
                    for (ox, oy) in opponent_offsets:
                        #print ("X = %d, Y = %d -> OX = %d, OY = %d"%(x, y, ox, oy))
                        paths = self.find_path_to(x, y, ox, oy, self._turn)
                        if paths:
                            #print ("X = %d, Y = %d -> OX = %d, OY = %d"%(x, y, ox, oy))
                            result.append((x, y))
        return result
    
    def find_path_to(self, x, y, offset_x, offset_y, colour):
        path = []
        while self.on_board((x + offset_x), (y + offset_y)):
            x = x + offset_x
            y = y + offset_y
            path.append((x, y))
            if self.get_at(x, y) == EMPTY:
                return None
            if self.get_at(x, y) == colour:
                return path
        return None
    
    def draw(self):
        valid_placements = self.all_valid_placements()
        (complete, whitepoints, blackpoints, empty) = self.stats()
        display = '  01234567 -> %ss turn (complete: %d, white: %d, black: %d, empty: %d)\n'%(self._turn, complete, whitepoints, blackpoints, empty)
        for y in range(HEIGHT):
            display += '%d|'%(y)
            for x in range(WIDTH):
                if self._board[(x * WIDTH) + y] == WHITE:
                    display += WHITE_X
                elif self._board[(x * WIDTH) + y] == BLACK:
                    display += BLACK_X
                elif (x, y) in valid_placements:
                    display += VALID_X
                else:
                    display += EMPTY_X
            display += '\n'
        return display
        
if __name__ == "__main__":
    # Random play
    import random
    import time

    start = time.time()
    iterations = 100
    
    for iteration in range(iterations):
        game = Reversi()
        (complete, whitepoints, blackpoints, empty) = game.stats()
        while not complete:
            options = game.all_valid_placements()
            if len(options) == 0:
                game.skip()
            else:
                (chosenx, choseny) = options[random.randint(0, len(options) - 1)]
                game.place(chosenx, choseny)
            (complete, whitepoints, blackpoints, empty) = game.stats()
            #print (game.draw())
        print ("White: %d, Black: %d (%d moves)"%(whitepoints, blackpoints, len(game._moves)))
        result = 'd'
        if whitepoints > blackpoints:
            result = 'w'
        if whitepoints < blackpoints:
            result = 'b'
        for (who, (x, y), board) in game._moves:
            #print ("  (winner was %s) %s played move -> x: %d, y: %d -> %s"%(result, who, x, y, board))
            pass
    
    end = time.time()
    print("Took %0.3fs to run %d iterations"%(end - start, iterations))

White: 22, Black: 42 (60 moves)
White: 34, Black: 30 (60 moves)
White: 33, Black: 31 (60 moves)
White: 54, Black: 10 (60 moves)
White: 46, Black: 18 (60 moves)
White: 34, Black: 30 (60 moves)
White: 21, Black: 42 (59 moves)
White: 31, Black: 33 (60 moves)
White: 18, Black: 46 (60 moves)
White: 25, Black: 39 (60 moves)
White: 37, Black: 27 (60 moves)
White: 27, Black: 37 (60 moves)
White: 28, Black: 36 (60 moves)
White: 25, Black: 39 (60 moves)
White: 39, Black: 25 (60 moves)
White: 39, Black: 25 (60 moves)
White: 32, Black: 32 (60 moves)
White: 33, Black: 31 (60 moves)
White: 19, Black: 45 (60 moves)
White: 33, Black: 31 (60 moves)
White: 32, Black: 32 (60 moves)
White: 30, Black: 34 (60 moves)
White: 21, Black: 43 (60 moves)
White: 47, Black: 17 (60 moves)
White: 49, Black: 15 (60 moves)
White: 24, Black: 40 (60 moves)
White: 31, Black: 33 (60 moves)
White: 35, Black: 29 (60 moves)
White: 33, Black: 31 (60 moves)
White: 46, Black: 18 (60 moves)
White: 27, Black: 37 (60 moves)
White: 3